# Project 1: ResNet34 vs VGG16 PGD L2 and LInf Attacks

In [13]:
!pip install torchattacks

In [14]:
import json
import time
import os
from PIL import Image

import torch
import torchvision.utils
from torchvision import models
import torchvision.transforms as transforms

import torchattacks

In [15]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

In [16]:
# An array with image file paths and their corresponding output label
# (valid with both VGG16 BN and ResNet34)
datalabels = json.load(open("./data/file2label.json"))

In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Works with both VGG16 BN and ResNet34
    transforms.ToTensor(),
])

## 1. Load ImageNet2012 Validation Data

In [18]:
class ImageNet2012Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, root, transform, datalabels):
        'Initialization'
        self.root = root
        self.transform = transform
        self.datalabels = datalabels

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.datalabels)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        datalabel = self.datalabels[index]

        # Load data and get label
        image_path = os.path.join(self.root, datalabel[0])
        image = Image.open(image_path).convert("RGB")
        imaget = self.transform(image)
        label = datalabel[1]

        return imaget, label

In [19]:
imagnet_data = ImageNet2012Dataset(root='./data/imagenet', transform=transform, datalabels=datalabels)
data_loader = torch.utils.data.DataLoader(imagnet_data, batch_size=10, shuffle=False)

## 2. Load ResNet34 or VGG16

In [20]:
#model = models.vgg16_bn(pretrained=True).to(device)
model = models.resnet34(pretrained=True).to(device)
model = model.eval()

## 3. Projected Gradient Descent (PGD) Attack

Specifications:
- L2 and LInf
- Epsilon = 2..10
- Steps = 2\*Epsilon
- Step = 1 step per iter

In [21]:
atks = [
        torchattacks.PGD(model, eps=2/255, alpha=1/255, steps=4),
        torchattacks.PGDL2(model, eps=2/255, alpha=1/255, steps=4),
    
        torchattacks.PGD(model, eps=3/255, alpha=1/255, steps=6),
        torchattacks.PGDL2(model, eps=3/255, alpha=1/255, steps=6),
    
        torchattacks.PGD(model, eps=4/255, alpha=1/255, steps=8),
        torchattacks.PGDL2(model, eps=4/255, alpha=1/255, steps=8),
    
        torchattacks.PGD(model, eps=5/255, alpha=1/255, steps=10),
        torchattacks.PGDL2(model, eps=5/255, alpha=1/255, steps=10),
    
        torchattacks.PGD(model, eps=6/255, alpha=1/255, steps=12),
        torchattacks.PGDL2(model, eps=6/255, alpha=1/255, steps=12),
    
        torchattacks.PGD(model, eps=7/255, alpha=1/255, steps=14),
        torchattacks.PGDL2(model, eps=7/255, alpha=1/255, steps=14),
    
        torchattacks.PGD(model, eps=8/255, alpha=1/255, steps=16),
        torchattacks.PGDL2(model, eps=8/255, alpha=1/255, steps=16),
    
        torchattacks.PGD(model, eps=9/255, alpha=1/255, steps=18),
        torchattacks.PGDL2(model, eps=9/255, alpha=1/255, steps=18),
    
        torchattacks.PGD(model, eps=10/255, alpha=1/255, steps=20),
        torchattacks.PGDL2(model, eps=10/255, alpha=1/255, steps=20),
       ]

In [22]:
def linf_distance(images, adv_images):
    diff = abs(images-adv_images.cpu())
    maxlist = []
    for i in diff:
        maxlist.append(torch.max(i))
    return 255*((sum(maxlist)/len(maxlist)).item())

In [23]:
def l2_distance(corrects, images, adv_images, device="cuda"):
    delta = (adv_images - images.to(device)).view(len(images), -1)
    l2 = torch.norm(delta[~corrects], p=2, dim=1).mean()
    return l2.item()

In [24]:
for atk in atks :
    
    print("-"*100)
    print(atk)
    
    correct = 0
    total = 0
    
    start = time.time()
    
    for images, labels in data_loader:
        
        adv_images = atk(images, labels)
        labels = labels.to(device)
        outputs = model(adv_images)

        _, pre = torch.max(outputs.data, 1)
        
        total += len(labels)
        corrects = (pre == labels)
        correct += corrects.sum()

        l2 = l2_distance(corrects, images, adv_images, device=device)
        linf = linf_distance(images, adv_images)
        print("Images: " + str(total) + " | Robust Acc: " + str(100* float(correct)/float(total)) + "% | L2 Distance: " + str(l2) + " | Linf Distance: " + str(linf) + "px")

    print('Total elapsed time (sec) : %.2f' % (time.time() - start))
    print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

----------------------------------------------------------------------------------------------------
PGD(model_name=ResNet, training=False, device=cuda:0, eps=0.00784313725490196, alpha=0.00392156862745098, steps=4, random_start=False, attack_mode=default, return_type=float)
Images: 10 | Robust Acc: 0.0% | L2 Distance: 2.4137461185455322 | Linf Distance: 2.0000074803829193px
Images: 20 | Robust Acc: 0.0% | L2 Distance: 2.4465858936309814 | Linf Distance: 2.0000074803829193px
Images: 30 | Robust Acc: 0.0% | L2 Distance: 2.4570069313049316 | Linf Distance: 2.0000074803829193px
Images: 40 | Robust Acc: 0.0% | L2 Distance: 2.42997670173645 | Linf Distance: 2.0000074803829193px
Images: 50 | Robust Acc: 0.0% | L2 Distance: 2.436323642730713 | Linf Distance: 2.0000074803829193px
Images: 60 | Robust Acc: 0.0% | L2 Distance: 2.428173065185547 | Linf Distance: 2.0000074803829193px
Images: 70 | Robust Acc: 0.0% | L2 Distance: 2.406029462814331 | Linf Distance: 2.0000070054084063px
Images: 80 | Ro

KeyboardInterrupt: 